------------------------------------------------------------------------------------------------------------------------------------------------------

# SCINT (for begginers)

This is a python library to process and analyse raw data from the IR02 lab.

Access the [DOCUMENTATION](https://scint.readthedocs.io/en/latest/index.html) for more information.

You have cloned the repository into a local directory and you are ready to start. 

### PLEASE! When you are done with the tutorial, move to the ```notebooks``` folder to start your own analysis 

In this way, you will be able to keep the tutorial as a reference for future use (or others to use it).

------------------------------------------------------------------------------------------------------------------------------------------------------

- - -
## STEP 0: prepare the environment

- - -
1. INSTALL:
- **VSCode with extensions (Remote-SSH, Jupyter, vscode-numpy-viewer, Python Environment Manager)**
- python + pip3

2. VIRTUAL ENVIROMENT to install all the packages we need there:
- **VSCode venv extension. It will recomend you the ```/scripts/requirements.txt``` packages and it will automatially install them :)**

- At CIEMAT:
    ```bash
    mkdir venv_python3.7
    cd venv_python3.7
    /cvmfs/sft.cern.ch/lcg/releases/Python/3.7.3-f4f57/x86_64-centos7-gcc7-opt/bin/python3 -m venv .
    source bin/activate
    ```

    ACTIVATE THE VENV and install the packages needed for the macros:
    ```bash
    source venv_python3.7/bin/activate
    cd SCINT/scripts
    sh setup.sh 
    ```
- - -

For start running the code you can:

### 1) FROM CIEMAT: connect to gaeuidc1.ciemat.es and run with the configuration set in ```input/TUTORIAL.txt``` (i.e. your data will be stored in ```/pc/choozdsk01/palomare/SCINT/TUTORIAL/```)

2) Less optimal option is to have the data locally

    a) Download the data
    ```bash
    sh copy_data.sh AFS_USER
    ```

    b) Mount in an empty data folder the ```TUTORIAL```:
    ```bash
    sshfs AFS_USER@pcaeXYZ.ciemat.es:/pc/choozdsk01/palomare/SCINT/TUTORIAL/ ../data
    ```
- - -


## STEP 1: understand the library

Once you have setup the environment, you can start to play with the library. You can see different folders:
```bash
00TUTORIAL -> this folder contains a tutorial notebook (this one) and template notebooks you can use as reference
input      -> input files needed to run the macros
lib        -> all the functions of the library itself
macros     -> macros to run the analysis. The are sorted by order of execution
notebooks  -> notebooks to run the analysis, PREPARED TO BE CHANGED AND NOT UPDATED IN THE REPOSITORY
```

Everytime you run a macro you can choose two options: 
1) Run it in the `guide` mode and the macro will ask you for the input variables needed (see an example for the input file)

<img src="images/starting_macro.png" alt="start"/>

2) Once you are sure about the variables to use you can run the macro with the defined flags:

```python
-h or --help
-i or --input_file
-l or --load_preset
-s or --save_preset
-k or --key
-v or --variables
-r or --runs
-c or --channels
-f or --filter
-d or --debug
```

### **Time to start the tutorial!**

----

## STEP 2: process the raw data


```bash
cd macros
python3 00Raw2Np.py        (-i TUTORIAL -d y)
```
<img src="images/00Raw2Np.png" alt="Raw2Np"/>


```bash
python3 01PreProcess.py     (-i TUTORIAL -k RawADC -d y)
```
<img src="images/01PreProcess.png" alt="PreProcess"/>

```bash
python3 02AnaProcess.py     (-i TUTORIAL -k AnaADC -d y)
```
<img src="images/02Process.png" alt="Process"/>

```bash
python3 03Integration.py     (-i TUTORIAL -k RawADC -d y)
```

<img src="images/03Integration.png" alt="Integration"/>

## STEP 2: visualize data

In [ ]:
# The first line in every macro should be the following for importing all the packages
import sys; sys.path.insert(0, '../'); from lib import *;
figure_features()

In [ ]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,                # Time in microseconds (True/False)
    "NORM":        False,               # Runs can be displayed normalised (True/False)
    "ALIGN":       True,                # Aligns waveforms in peaktime (True/False)
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",                  # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,               # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",    # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,               # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "SHOW":        True
    }

The following cell can be used to load the configuration file.
Check that everything is OK and choose your runs!

In [ ]:
info = read_input_file("TUTORIAL")           # Read input file

calib_runs     = [ 1]                        # Calibration runs
laser_runs     = [ 9]                        # Laser runs
alpha_runs     = [25]                        # Alpha runs
muons_runs     = [29]                        # Muons runs
noise_runs     = [128]                       # Noise runs

channels = [0,6]

for key in info: print(key,info[key])  # Print loaded info
for key in info: print(key,info[key])  # Print loaded info

In [ ]:
# LOAD YOUR RUNS

my_runs_calib = load_npy(np.asarray(calib_runs).astype(int),np.asarray(channels).astype(int),preset="EVA",info=info,compressed=True)
my_runs_laser = load_npy(np.asarray(laser_runs).astype(int),np.asarray(channels).astype(int),preset="EVA",info=info,compressed=True)
my_runs_alpha = load_npy(np.asarray(alpha_runs).astype(int),np.asarray(channels).astype(int),preset="EVA",info=info,compressed=True)
my_runs_muons = load_npy(np.asarray(muons_runs).astype(int),np.asarray(channels).astype(int),preset="RAW",info=info,compressed=True)
my_runs_noise = load_npy(np.asarray(noise_runs).astype(int),np.asarray(channels).astype(int),preset="EVA",info=info,compressed=True)

time = np.arange(len(my_runs_calib[calib_runs[0]][channels[0]]["AnaAveWvf"][0]))*4e-9

### 2.1 INDIVIDUAL EVENTS

By running the next function you can visualize the RAW events of the selected run to see everything is OK.

In [ ]:
my_runs = my_runs_muons
runs    = muons_runs
chs     = channels

EVENT2PLOT = np.random.randint(len(my_runs[runs[0]][chs[0]]["RawADC"]))

nch=len(chs); ncol=int((nch+1)/2); nrow=int((nch)/2); z=0; axs = []
if nch < 4: 
        fig, ax = plt.subplots(nch ,1, figsize = (10,8))
        axs = ax.T.flatten(); 
        if nch == 1: axs.append(ax)
        else: axs = ax
else: fig, ax = plt.subplots(dpi=200,ncols= ncol,nrows=nrow,figsize=[8,3],sharex=True)

time = np.arange(len(my_runs[runs[0]][chs[0]]["RawADC"][EVENT2PLOT]))*my_runs[runs[0]][chs[0]]["Sampling"]
for c,ch in enumerate(chs):
        i = int(z/ncol)
        j = z%ncol

        RawADC  = my_runs[runs[0]][ch]["RawADC"]    [EVENT2PLOT]
        PedSTD  = my_runs[runs[0]][ch]["RawPedSTD"] [EVENT2PLOT]
        PedMean = my_runs[runs[0]][ch]["RawPedMean"][EVENT2PLOT]
        ax[c].tick_params(axis='both', which='major', labelsize=5)
        ax[c].grid()
        ax[c].plot( time,  RawADC,linewidth=.5, label="Ch: "+str(ch))
        ax[c].plot([0, time[-1]], [PedMean,PedMean],                   color="k"  ,linewidth=.7)
        ax[c].plot([0, time[-1]], [PedMean+PedSTD,PedMean+PedSTD],"--",color="k"  ,linewidth=.7)
        ax[c].plot([0, time[-1]], [PedMean-PedSTD,PedMean-PedSTD],"--",color="k"  ,linewidth=.7)
        ax[c].legend()  
        ax[c].set_title("Run {} - Ch {} - Event Number {}".format(my_runs["NRun"][0],ch,EVENT2PLOT),size = 14)
        ax[c].xaxis.offsetText.set_fontsize(14) # Smaller fontsize for scientific notation
        ax[c].grid(True, alpha = 0.7)

        del RawADC,PedSTD,PedMean
        z+=1;   

fig.supylabel("ADC Counts")
fig.supxlabel(r'Time [s]')
# fig.supxlabel(r'Time [$\mu$s]')

plt.show()

Check also ``0XVisEvents.ipynb`` for more options.

There are diferent types of acquired runs (see input file) and therefore you will need different visualization for them

### 2.2 NOISE 


In [ ]:
noise = my_runs_noise[noise_runs[0]][channels[0]]["AnaAveWvf"][0]
sp    = np.fft.rfft(noise)
freq  = np.fft.rfftfreq(len(time), d = 4e-9)

fig = px.line(x=freq, y=sp.imag)
custom_plotly_layout(fig, title='FFT', xaxis_title='Frequency [Hz]', yaxis_title='FFT').show()


Check also ``Noise.ipynb``

### 2.3 LASER AND ALPHA

In [ ]:
LASER = []
for c,ch in enumerate(channels):
    laser = my_runs_laser[laser_runs[0]][channels[c]]["AnaAveWvf"][0]
    laser = laser / np.max(laser)
    
    ref_max_idx = np.argmax(laser)
    idx_move    = np.argmax(laser)
    laser       = shift(laser, ref_max_idx-idx_move, cval = 0)

    LASER.append(laser)
    
df_laser = pd.DataFrame({"Time":time,"Laser_ch0":LASER[0],"Laser_ch6":LASER[1]})

In [ ]:
fig_laser = px.line(df_laser, x="Time", y=["Laser_ch0","Laser_ch6"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_laser.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_laser,["Laser (SiPM)","Laser (XA)"])
custom_plotly_layout(fig_laser,"Time","Normalized ADC","Laser signals").show()

In [ ]:
ALPHA = []
for c,ch in enumerate(channels):
    alpha = my_runs_alpha[alpha_runs[0]][channels[c]]["AnaAveWvf"][0]
    alpha = alpha / np.max(alpha)

    ref_max_idx = np.argmax(alpha)
    idx_move    = np.argmax(alpha)
    alpha       = shift(alpha, ref_max_idx-idx_move, cval = 0)
    ALPHA.append(alpha)


df_alpha = pd.DataFrame({"Time":time,"Alpha_ch0":ALPHA[0],"Alpha_ch6":ALPHA[1]})

In [ ]:
fig_alpha = px.line(df_alpha, x="Time", y=["Alpha_ch0","Alpha_ch6"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_alpha.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_alpha,["Alpha (SiPM)", "Alpha (XA)"])
custom_plotly_layout(fig_alpha,"Time","Normalized ADC","Alpha signals").show()

Check also ``0WVisWvf.ipynb``

In the different ```notebooks``` you can find ideas and templates for plots you can use to make your analysis.